In this notebook we give a proof of concept of unitary compiling using TFQ. 

In [1]:
%load_ext autoreload
%autoreload 2

import sympy 
import numpy as np 
import pandas as pd 
import tensorflow as tf
from utilities.circuit_database import CirqTranslater
from utilities.templates import *
from utilities.variational import Minimizer
from utilities.misc import get_qubits_involved, reindex_symbol, shift_symbols_down
import matplotlib.pyplot as plt 
import tensorflow_quantum as tfq
import cirq
from utilities.compiling import *
from utilities.simplifier import Simplifier
from utilities.discrimination import *
from utilities.idinserter import IdInserter
from utilities.evaluator import Evaluator
from utilities.gate_killer import GateKiller

In [84]:
np.random.seed(0)

translator = CirqTranslater(3, untouchable_blocks = [1])

translator.env_qubits = [2]
translator.encoder_id = 0
translator.channel_id = 1
translator.decoder_id = 2


simplifier = Simplifier(translator)


etas = [0.01, 1.]
minimizer = Minimizer(translator, mode="discrimination", params=etas)

killer = GateKiller(translator, mode="discrimination", params = etas)


inserter = IdInserter(translator.n_qubits, untouchable_blocks=translator.channel_id)


args_evaluator = {"n_qubits":translator.n_qubits, "problem":"acd","params":etas}
evaluator = Evaluator(args=args_evaluator, lower_bound_cost=minimizer.lower_bound_cost, nrun=0)


In [85]:
u1_layer_encoder = u1_layer(translator, inds= [0,1], block_id=translator.encoder_id)
channel_db = amplitude_damping_db(translator, qubits_ind=[1,2], eta=1, block_id = translator.channel_id)
u1_layer_decoder = u1_layer(translator, inds = [0,1], block_id=translator.decoder_id)

u2_layer_encoder = u2_db(translator, 0,1, block_id=translator.encoder_id)
channel_db = amplitude_damping_db(translator, qubits_ind=[1,2], eta=1, block_id = translator.channel_id)
u2_layer_decoder = u2_db(translator, 0,1, block_id=translator.decoder_id)


circuit_db = concatenate_dbs([u1_layer_encoder, channel_db, u1_layer_decoder])
#circuit_db = concatenate_dbs([u2_layer_encoder, channel_db, u2_layer_decoder])

circuit, circuit_db = translator.give_circuit(circuit_db)
circuit

(0, 0): ───Rz(th_0)───Rx(th_1)───Rz(th_2)───Rz(th_11)───Rx(th_12)───Rz(th_13)─────────────────────────────────────────────────────────────────

(0, 1): ───Rz(th_3)───Rx(th_4)───Rz(th_5)───H───────────X────────────────────────X───H────────────────X───Rz(th_14)───Rx(th_15)───Rz(th_16)───
                                                        │                        │                    │
(0, 2): ───Rz(th_6)───Rx(th_7)───Rz(th_8)───H───────────@───────────Ry(0.318π)───@───H───Ry(0.318π)───@───Rz(th_17)───Rx(th_18)───Rz(th_19)───

In [61]:
batch_circuits, trainable_symbols, trainable_params_value = prepare_optimization_discrimination(translator, circuit_db, etas)
cost, resolver, training_history = minimizer.minimize(batch_circuits, symbols = trainable_symbols, parameter_values = trainable_params_value )
circuit_db = translator.update_circuit_db_param_values(circuit_db, resolver)

In [62]:
circuit_db["symbol"]

0      th_0
1      th_1
2      th_2
3      th_3
4      th_4
5      th_5
6      th_6
7      th_7
8      th_8
9      None
10     None
11     None
12     th_9
13     None
14     None
15     None
16    th_10
17     None
18    th_11
19    th_12
20    th_13
21    th_14
22    th_15
23    th_16
24    th_17
25    th_18
26    th_19
Name: symbol, dtype: object

In [63]:
original_db = circuit_db.copy()

In [64]:
block_db = circuit_db[circuit_db["block_id"] == 2]
block_db_trainable = block_db[block_db["trainable"] == True]
block_db_trainable = block_db_trainable[~block_db_trainable["symbol"].isna()]
block_db_trainable = block_db_trainable[~block_db_trainable["symbol"].isna()]
all_candidates = list(block_db_trainable.index)

In [67]:
gates_on_qubit, on_qubit_order = killer.get_positional_dbs(block_db_trainable)

In [69]:
from utilities.misc import qubit_get

In [70]:
all_candidates[0]

21

In [78]:
qubit_get(block_db_trainable.loc[all_candidates[3]]["ind"], translator)

2

In [39]:
tok = original_db.copy()

In [44]:
tok = tok.drop(labels=[11])

In [47]:
original_db.loc[11]

ind                  5
symbol            None
param_value        NaN
trainable        False
block_id             1
channel_param      NaN
Name: 11, dtype: object

In [79]:
circuit_db = original_db.copy()

In [82]:
evaluator.add_step(circuit_db, cost, relevant=True)
evaluator.save_dicts_and_displaying()

circuit_db, cost = killer.remove_irrelevant_gates(cost, circuit_db)
evaluator.add_step(circuit_db, cost, relevant=False)
evaluator.save_dicts_and_displaying()


kill 1qbit gate, try 0/12. Increased by: 7.270030550898809e-07%
kill 1qbit gate, try 1/12. Increased by: 7.270030550898809e-07%
kill 1qbit gate, try 2/12. Increased by: 0.0%
kill 1qbit gate, try 3/12. Increased by: 0.0%


In [6]:
mutated_circuit_db = inserter.insert_many_mutations(circuit_db )
simplified_db, nreds = simplifier.reduce_circuit(mutated_circuit_db)

batch_circuits, trainable_symbols, trainable_params_value = prepare_optimization_discrimination(translator, simplified_db, etas)

In [10]:
circuit_db["symbol"]

0      th_0
1      th_1
2      th_2
3      th_3
4      th_4
5      th_5
6      None
7      None
8      th_5
9      None
10     None
11     None
12     th_6
13     None
14     th_7
15     th_8
16     th_9
17    th_10
18    th_11
19    th_12
20    th_13
21    th_14
22    th_19
Name: symbol, dtype: object

In [9]:
mutated_circuit_db["symbol"]

0      th_0
1      th_1
2      th_2
3      th_3
4      th_4
5      th_5
6      th_6
7      th_7
8      th_8
9      th_9
10    th_10
11    th_11
12     None
13     None
14    th_11
15     None
16     None
17     None
18    th_12
19     None
20    th_13
21    th_14
22    th_15
23    th_16
24    th_17
25    th_18
26    th_19
27    th_20
28    th_21
29    th_22
30    th_23
31    th_28
Name: symbol, dtype: object

In [ ]:

    cost, resolver, training_history = minimizer.minimize(batch_circuits, symbols = trainable_symbols, parameter_values = trainable_params_value )
    mutation_db = translator.update_circuit_db_param_values(mutation_db, resolver)

In [14]:

    

    if evaluator.accept_cost(cost):
        circuit_db = mutation_db
        circuit_db, cost = killer.remove_irrelevant_gates(cost, circuit_db)
        evaluator.add_step(circuit_db, cost, relevant=False)
        evaluator.save_dicts_and_displaying()

vans iter 0


ValueError: All elements of symbol_names must be unique.

In [15]:
mutated_circuit_db

,ind,symbol,param_value,trainable,block_id,channel_param
0,6,th_0,-9.576239,True,0,NaN
1,9,th_1,3.407127,True,0,NaN
2,6,th_2,2.210824,True,0,NaN
3,7,th_3,0.169105,True,0,NaN
4,10,th_4,-15.707957,True,0,NaN
5,7,th_5,2.666668,True,0,NaN
6,11,th_6,-5.722386,True,0,NaN
7,8,th_7,7.853981,True,0,NaN
8,16,None,NaN,False,1,NaN
9,17,None,NaN,False,1,NaN
